In [129]:
import pandas as pd
from glob import glob
import re
import math
import numpy as np


In [130]:
def load_data(path):
    data = pd.read_csv(path, names=['name1', 'name2', 'lr_pred', 'rl_pred', 'lr_true', 'rl_true']) 
    data['off'] = abs(data['rl_true']-data['rl_pred']) + abs(data['lr_true']-data['lr_pred'])
    data['off_sqrt'] = data['off'].pow(2)
    data['total'] = data['rl_true'] + data['lr_true']
    data['perc_off'] = data['off']/data['total']
    return data

data = load_data('results/default_tensorflow/results.txt')
print(data['off_sqrt'].sum(), data['off'].mean())

data = load_data('results/default_pytorch_finetuned/results.txt')
print(data['off_sqrt'].sum(), data['off'].mean())

data = load_data('results/new_regions/results.txt')
data.groupby(['name1'])['off_sqrt'].sum()
print(data['off_sqrt'].sum(), data['off'].mean())

3101.89759866 5.91496395826
3421.40698832 6.18560127303
3173.16069346 6.02813266106


In [136]:
frames = []
best_off = 10000
best_off_conf = None

best_sqrt = 10000
best_sqrt_conf = None
for path in glob('results/v2*'):
    r = re.search(r'height([0-9]+)_width([0-9]+)', path)
    data = load_data('{}/results.txt'.format(path))
    data['height'] = int(r.group(1))/10.
    data['width'] = int(r.group(2))/10.
    
    if data['off'].mean() < best_off:
        best_off_conf = (int(r.group(1))/10., int(r.group(2))/10.)
        best_off = data['off'].mean()
    
    print(int(r.group(1))/10., int(r.group(2))/10., data['off'].mean())
    frames.append(data)
data = pd.concat(frames).reset_index(drop=True)

del frames
pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', '{:,.2f}'.format)

print(best_off, best_off_conf)

#print(data)

1.8 1.0 6.8678174197
1.5 5.0 6.09726504801
0.8 3.0 6.07217178445
1.5 1.0 6.19085686673
1.0 1.0 6.19801836733
0.6 1.5 6.46577711255
0.8 5.0 5.99108765026
1.0 1.5 6.12351806031
1.0 4.0 6.08661505589
0.6 5.0 6.37347524368
0.8 1.5 6.03095963474
1.8 5.0 6.50040257655
0.6 4.0 6.547646991
0.6 2.0 6.41599900401
1.5 1.5 6.13358850216
1.8 4.0 7.17283455962
1.8 3.0 6.99934066933
1.0 2.0 6.10487714083
0.4 1.5 6.86332665423
0.6 1.0 6.60311517009
0.4 2.0 6.7718894981
1.0 5.0 6.00823351171
1.5 2.0 6.19150206773
0.4 3.0 6.85705083599
1.8 1.5 6.69551023082
1.2 4.0 6.06197090636
0.6 3.0 6.44752497238
1.8 2.0 6.76484924869
1.5 3.0 6.18300707222
1.5 4.0 6.57620109465
0.8 2.0 5.99288562928
0.4 5.0 6.70933178522
0.4 4.0 6.87215689591
0.8 4.0 6.0408279543
0.8 1.0 6.11667606775
1.0 3.0 6.02813172121
5.99108765026 (0.8, 5.0)


In [132]:
minimals = data.loc[data.groupby(['name1','name2'])['off'].idxmin()]
display(minimals.groupby(['name1','name2']).min()[['perc_off', 'off_sqrt','height','width']])

perc_off    off_sqrt  height  width
name1 name2                                     
IM01  0      0.090095    1.371790     1.0    1.0
      1      0.001302    0.000550     1.0    2.0
IM02  0      0.021185    0.044882     0.8    1.0
      1      0.005424    0.001059     0.4    2.0
      2      0.054099    0.749225     1.2    4.0
      3      0.004080    0.008059     1.5    4.0
IM03  0      0.013042    0.208373     0.8    1.0
      1      0.004603    0.000763     1.0    4.0
      2      0.144216   17.491439     1.8    3.0
IM04  0      0.408007   88.062584     0.8    5.0
      1      0.179482    8.246704     1.8    3.0
      2      0.491573    8.699183     0.4    1.5
      3      0.574029  118.952906     1.8    2.0
IM05  0      0.009940    0.121028     0.4    1.5
      1      0.180683   54.878801     1.5    5.0
      2      0.004400    0.034154     1.5    2.0

In [139]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', '{:,.2f}'.format):
    display(data.groupby(['height', 'width']).mean()['off'])

height  width
0.40    1.50    6.86
        2.00    6.77
        3.00    6.86
        4.00    6.87
        5.00    6.71
0.60    1.00    6.60
        1.50    6.47
        2.00    6.42
        3.00    6.45
        4.00    6.55
        5.00    6.37
0.80    1.00    6.12
        1.50    6.03
        2.00    5.99
        3.00    6.07
        4.00    6.04
        5.00    5.99
1.00    1.00    6.20
        1.50    6.12
        2.00    6.10
        3.00    6.03
        4.00    6.09
        5.00    6.01
1.20    4.00    6.06
1.50    1.00    6.19
        1.50    6.13
        2.00    6.19
        3.00    6.18
        4.00    6.58
        5.00    6.10
1.80    1.00    6.87
        1.50    6.70
        2.00    6.76
        3.00    7.00
        4.00    7.17
        5.00    6.50
Name: off, dtype: float64